# Maps

## Agenda

1. Discussion: pros/cons of array-backed and linked structures
2. `set` and `dict` lookup performance
3. The **Map** ADT
4. Hashing and Hashcodes
5. Hashtables
    - Collisions and the "Birthday problem"
6. Runtime analysis & Discussion

## 1. Discussion: pros/cons of array-backed and linked structures

Between the array-backed and linked list we have:

1. $O(1)$ indexing (array-backed)
2. $O(1)$ appending (array-backed & linked)
3. $O(1)$ insertion/deletion without indexing (linked)
4. $O(N)$ linear search (unsorted)
5. $O(\log N)$ binary search, when sorted (only array-backed lists)

Lists inherently prioritize position-based access due to their sequential nature -- array-backed lists are optimized for direct indexing, while linked lists are excellent for cursor-driven operations.

But what if we would like to prioritize search?

## 2. `set` and `dict` lookup performance

The `set` and `dict` types don't support positional access (i.e., by index), but do support lookup/search. Just how fast are they?

In [ ]:
# let's use linear and binary search as baselines for comparison

def lin_search(lst, x):
    for y in lst:
        if x == y:
            return True
    else:
        return False
    
def bin_search(lst, x):
    # assumes lst is sorted
    low = 0
    hi  = len(lst)-1
    while low <= hi:
        mid = (low + hi) // 2
        if x < lst[mid]:
            hi  = mid - 1
        elif x < lst[mid]:
            low = mid + 1
        else:
            return True
    else:
        return False

In [ ]:
import timeit
import matplotlib.pyplot as plt
import numpy as np

%matplotlib inline
plt.rcParams['figure.figsize'] = [10, 6] # set size of plot

ns = np.linspace(100, 10_000, 50, dtype=int)

to_find = -1

ts_linsearch = [timeit.timeit(f'lin_search(lst, lst[{to_find}])',
                              setup=f'lst = list(range({n}))',
                              globals=globals(),
                              number=100)
                for n in ns]

ts_binsearch = [timeit.timeit(f'bin_search(lst, {to_find})',
                              setup=f'lst = list(range({n}))',
                              globals=globals(),
                              number=100)
                for n in ns]

ts_setsearch = [timeit.timeit(f'{to_find} in st', 
                              setup=f'st = set(range({n}))',
                              globals=globals(),
                              number=100)
                for n in ns]

ts_dctsearch = [timeit.timeit(f'{to_find} in dct',
                              setup=f'dct = {{x:x for x in range({n})}}',
                              globals=globals(),
                              number=100)
                for n in ns]

plt.plot(ns, ts_linsearch, 'sr')
plt.plot(ns, ts_binsearch, 'sg')
plt.plot(ns, ts_setsearch, 'ob')
plt.plot(ns, ts_dctsearch, 'om');

Somehow, by discarding positional access, sets and dictionaries appear to be able to implement search in **constant time**!

How is this magic possible?

## 3. The **Map** ADT

We will focus next on the "*map*" abstract data type (aka "associative array" or "dictionary"), which is used to associate keys (which must be unique) with values. 

A map *does not* intrinsically impose any ordering on its contents --- i.e., an implementation of a map does not need to support positional access to keys, nor report a consistent view of key order.

Python's `dict` type is an implementation of the map ADT. 

### 3.1. But what about sets?

Given an implementation of a map, it should be straightforward to use it to implement a set:

In [ ]:
class MySet:
    def __init__(self):
        self.dct = dict()
    
    def add(self, value):
        pass
    
    def __contains__(self, value):
        pass
        
    def intersection(self, other):
        assert isinstance(other, MySet)
        pass
    
    def __iter__(self):
        return iter(self.dct)
            
    def __repr__(self):
        return '{' + ', '.join(repr(x) for x in self) + '}'

In [ ]:
st = MySet()

for c in 'hello world!':
    st.add(c)

st

In [ ]:
set('hello world!')

In [ ]:
st2 = MySet()

for c in 'farewell planet':
    st2.add(c)
    
st.intersection(st2)

In [ ]:
set('hello world!') & set('farewell planet')

### 3.2. A simple map implementation

In [ ]:
class MapDS:
    def __init__(self):
        self.data = []
    
    def __setitem__(self, key, value):
        pass
    
    def __getitem__(self, key):
        pass
            
    def __contains__(self, key):
        pass

In [ ]:
m = MapDS()
m['batman'] = 'bruce wayne'
m['superman'] = 'clark kent'
m['spiderman'] = 'peter parker'

In [ ]:
m['batman']

In [ ]:
m['batman'] = 'tony stark'

In [ ]:
m['batman']

How can we make the leap from linear runtime complexity to constant?!

## 4. Hashing and Hashcodes

*Hashcodes* (a.k.a. hash values or just hashes) are simple numerical values (typically integers) computed for objects by *hash functions*. Their implementation is typically based on simple mathematical operations, such as multiplication, division, and modular arithmetic, applied to the object's data.

In [22]:
# a simple hash function
def myhash(s):
    h = 0
    for c in s:
        h = 31 * h + ord(c)
    return h

myhash('hello')

99162322

For our purposes, we will consider an object's hashcode to be akin to a "fingerprint" of its data. We will also assume that hashcodes are very efficient to compute (i.e., in constant time).

Python provides a built-in `hash()` function that returns an object's hashcode.

In [ ]:
hash('hello')

In [ ]:
hash('batman')

In [ ]:
hash('batmen') 

In [ ]:
[hash(s) for s in ['different', 'objects', 'have', 'very', 'different', 'hashes']]

How can we use hash codes to help us implement a mapping sttrucre with constant-time lookup?

### Random Hashing

The `hash` function in Python is *randomized* by default -- i.e., each time a Python interpreter is fired up, the implementation of `hash` will use a different "seed" for the random number generator used in computing hashes. While hashcodes computed for a given value will be consistent for a given interpreter instance, they will not be across instances! This means we shouldn't save hashcodes for values to disk, or save them to a database, as values will almost certainly hash to different hashcodes after we restart our software!

Why does Python do this? More later!

## 5. Hashtables

A **hashtable** is an implementation of the map ADT that uses the hashcode for a key to compute an index into an array where the corresponding key/value pair will be stored.

![](../images/10-ht-simple.png)

![](../images/10-ht-simple-contains.png)

In [ ]:
class Hashtable:
    def __init__(self, n_buckets):
        self.buckets = [None] * n_buckets
        
    def __setitem__(self, key, val):
        pass
    
    def __getitem__(self, key):
        pass
        
    def __contains__(self, key):
        try:
            _ = self[key]
            return True
        except:
            return False

In [ ]:
ht = Hashtable(100)
ht['spiderman'] = 'peter parker'
ht['batman'] = 'bruce wayne'
ht['superman'] = 'clark kent'

In [ ]:
ht['spiderman']

In [ ]:
ht['batman']

In [ ]:
ht['superman']

### 5.1. On Collisions

#### The "Birthday Problem"

Problem statement: Given $N$ people at a party, how likely is it that at least two people will have the same birthday (assuming a fixed 365 days/year)?

How many people do you think are needed for a 50% chance of a shared birthday? 99%?

The trick to answering the birthday problem is to consider the *complement* of it; i.e., the event of "no shared birthdays". The probability that at least two people share a birthday is $1$ minus the probability that *nobody shares a birthday*.

- For 1 person, the probability is $1 - 1 = 0$
- For 2 people, the probability is $1 - \frac{364}{365} = \frac{1}{365}$
- For 3 people, the probability is $1 - \frac{364}{365} \times \frac{363}{365}$
- For 4 people, the probability is $1 - \frac{364}{365} \times \frac{363}{365} \times \frac{362}{365}$
- For $N$ people, the probability is $1 - \frac{364}{365} \times \frac{363}{365} \times \ldots \times \frac{365 - N + 1}{365}$

In [ ]:
def birthday_p(n_people):
    p_inv = 1
    for n in range(365, 365-n_people, -1):
        p_inv *= n / 365
    return 1 - p_inv

In [ ]:
birthday_p(3)

In [ ]:
1-364/365*363/365

In [ ]:
n_people = range(1, 80)
plt.plot(n_people, [birthday_p(n) for n in n_people]);

#### General collision statistics
Repeat the birthday problem, but with a given number of values and "buckets" that are allotted to hold them. How likely is it that two or more values will map to the same bucket?

In [ ]:
def collision_p(n_values, n_buckets):
    p_inv = 1
    for n in range(n_buckets, n_buckets-n_values, -1):
        p_inv *= n / n_buckets
    return 1 - p_inv

In [ ]:
collision_p(23, 365) # same as birthday problem, for 23 people

In [ ]:
collision_p(10, 100)

In [ ]:
collision_p(100, 1000)

In [ ]:
# keeping number of values fixed at 100, but vary number of buckets: visualize probability of collision
n_buckets = range(100, 100001, 1000)
plt.plot(n_buckets, [collision_p(100, nb) for nb in n_buckets]);

In [ ]:
def avg_num_collisions(n, b):
    """Returns the expected number of collisions for n values uniformly distributed
    over a hashtable of b buckets. Based on (fairly) elementary probability theory.
    (Pay attention in MATH 474!)"""
    return n - b + b * (1 - 1/b)**n

In [ ]:
avg_num_collisions(28, 365)

In [ ]:
avg_num_collisions(1000, 1000)

In [ ]:
avg_num_collisions(1000, 10000)

### 5.2. Dealing with Collisions
To deal with collisions in a hashtable, we simply create a "chain" of key/value pairs for each bucket where collisions occur. The chain needs to be a data structure that supports quick insertion — natural choice: the linked list!

![](../images/10-ht-collisions.png)

![](../images/10-ht-contains.png)

In [ ]:
class Hashtable:
    class Node:
        def __init__(self, key, val, next=None):
            self.key = key
            self.val = val
            self.next = next
            
    def __init__(self, n_buckets=1000):
        self.buckets = [None] * n_buckets
        
    def __setitem__(self, key, val):
        bidx = hash(key) % len(self.buckets)
    
    def __getitem__(self, key):
        bidx = hash(key) % len(self.buckets)
    
    def __contains__(self, key):
        try:
            _ = self[key]
            return True
        except:
            return False

In [ ]:
ht = Hashtable(100)
ht['batman'] = 'bruce wayne'
ht['superman'] = 'clark kent'
ht['spiderman'] = 'peter parker'

In [ ]:
ht['batman']

In [ ]:
ht['superman']

In [ ]:
ht['spiderman']

In [ ]:
def init_ht(size):
    ht = Hashtable(size)
    for x in range(size):
        ht[x] = x
    return ht

ns = np.linspace(100, 10_000, 50, dtype=int)
ts_htsearch = [timeit.timeit(f'{0} in ht',
                             setup='ht = init_ht({})'.format(n),
                             globals=globals(),
                             number=100)
               for n in ns]

In [ ]:
plt.plot(ns, ts_binsearch, 'ro')
plt.plot(ns, ts_htsearch, 'gs')
plt.plot(ns, ts_dctsearch, 'b^');

### 5.3. Loose ends

#### Iteration

In [ ]:
class Hashtable(Hashtable):
    def __iter__(self):
        pass

In [ ]:
ht = Hashtable(100)
ht['batman'] = 'bruce wayne'
ht['superman'] = 'clark kent'
ht['spiderman'] = 'peter parker'

In [ ]:
for k in ht:
    print(k)

#### Key ordering

In [ ]:
ht = Hashtable()
d = {}
for x in 'apple banana cat dog elephant'.split():
    d[x[0]] = x
    ht[x[0]] = x

In [ ]:
for k in d:
    print(k, '=>', d[k])

In [ ]:
for k in ht:
    print(k, '=>', ht[k])

To implement an ordered hashtable, it is necessary to maintain a separate data structure to keep track of the order of keys (by insertion). Here's a clever way to do it -- you'll flesh it out in lab!

![](../images/10-ht-ordered.png)

### 5.5. Load factor & Rehashing

It is clear that the ratio of the number of keys to the number of buckets (known as the **load factor**) can have a significant effect on the performance of a hashtable.

A fixed number of buckets doesn't make sense, as it might be wasteful for a small number of keys, and also scale poorly to a relatively large number of keys. And it also doesn't make sense to have the user of the hashtable manually specify the number of buckets (which is a low-level implementation detail). 

Instead: a practical hashtable implementation would start with a relatively small number of buckets, and if/when the load factor increases beyond some threshold (typically 1), it *dynamically increases the number of buckets* (typically to twice the previous number). This requires that all existing keys be *rehashed* to new buckets (why?).

What is the runtime complexity of rehashing a hashtable of *N* keys?

### 5.6. Uniform hashing

Ultimately, the performance of a hashtable also heavily depends on hashcodes being *uniformly distributed* --- i.e., where, statistically, each bucket has roughly the same number of keys hashing to it. Designing hash functions that do this is an algorithmic problem that's outside the scope of this class!

## 6. Runtime analysis & Discussion

For a hashtable with $N$ key/value entries, in the worst imaginable scenario all keys hash to the same bucket, and we end up having to navigate through a chain of $N$ collisions when inserting/searching/deleting, which gives us the  following *worst-case runtime complexities*:

- Insertion: $O(N)$
- Lookup: $O(N)$
- Deletion: $O(N)$

BUT, if we assume uniform hashing and the rehashing behavior described above, it is possible to prove that hashtables have $O(1)$ *amortized (i.e., average) runtime complexity*. Proving this is also beyond the scope of this class (but is borne out by empirical data).

### Denial of Service Attacks and Random Hashing

Hashtables are unique in that they depend on good average case runtime complexity, and the incredibly low likelihood of a large number of collisions happening.

But what if someone knew what different values hashed to in advance, and intentionally caused a huge number of keys to be entered into a hashtable that would result in collisions?

This is the basis of a **denial of service attack** aimed at taking advantage of the worst-case runtime complexity of a Hashtable! The good news: Python makes this much more difficult by randomizing hashes by default -- but it also means you need to take care to guard your hash function if you implement one yourself.

## Vocabulary list

- hashtable
- hashing and hashes
- collision
- hash buckets & chains
- birthday problem
- load factor
- rehashing
- denial of service attack

---

## Addendum: On *Hashability*

Remember: *a given object must always hash to the same value*. This is required so that we can always map the object to the same hash bucket.

Hashcodes for collections of objects are usually computed from the hashcodes of its contents, e.g., the hash of a tuple is a function of the hashes of the objects in said tuple:

In [ ]:
hash(('two', 'strings'))

This is useful. It allows us to use a tuple, for instance, as a key for a hashtable.

However, if the collection of objects is *mutable* — i.e., we can alter its contents — this means that we can potentially change its hashcode.`

If we were to use such a collection as a key in a hashtable, and alter the collection after it's been assigned to a particular bucket, this leads to a serious problem: the collection may now be in the wrong bucket (as it was assigned to a bucket based on its original hashcode)!

For this reason, only immutable types are, by default, hashable in Python. So while we can use integers, strings, and tuples as keys in dictionaries, lists (which are mutable) cannot be used. Indeed, Python marks built-in mutable types as "unhashable", e.g.,

In [ ]:
hash([1, 2, 3])

That said, Python does support hashing on instances of custom classes (which are mutable). This is because the default hash function implementation does not rely on the contents of instances of custom classes. E.g.,

In [ ]:
class Student:
    def __init__(self, fname, lname):
        self.fname = fname
        self.lname = lname

In [ ]:
s = Student('John', 'Doe')
hash(s)

In [ ]:
s.fname = 'Jane'
hash(s) # same as before mutation

We can change the default behavior by providing our own hash function in `__hash__`, e.g.,

In [ ]:
class Student:
    def __init__(self, fname, lname):
        self.fname = fname
        self.lname = lname
        
    def __hash__(self):
        return hash(self.fname) + hash(self.lname)

In [ ]:
s = Student('John', 'Doe')
hash(s)

In [ ]:
s.fname = 'Jane'
hash(s)

But be careful: instances of this class are no longer suitable for use as keys in hashtables (or dictionaries), if you intend to mutate them after using them as keys!